# 최종 프로젝트
- 7일차 (10/28)


# 섹션3. 방화벽 pfsense 서버 구축 및 설정

# 방화벽 기초 이론 및 pfsense 설치

## 기초 이론

- 방화벽에 대한 기초 이론

### 방화벽 개념

- 네트워크에서의 방화벽은 신뢰하지 않는 외부 네트워크와 신뢰하는 내부 네트워크 사이를 지나는 패킷을 미리 정해놓은 규칙에 따라 차단하거나 보내주는 기능을 하는 하드웨어나 소프트웨어를 말한다.
- 침해사고를 당했다 하더라도 다른 네트워크로 쉽게 넘어가지 않도록 만들어주는 역할을 한다.

### 기능

- 접근 제어(Access Control)
    - 통과시킬 접근과 그렇지 않은 접근을 결정하여 허용과 차단을 함
    - 접근 제어 방식은 구현 방법에 따라 패킷 필터링 방식과 프록시 방식으로 나뉨
        - 패킷 필터링: 패킷이 지나갈 때 패킷을 확인
        - 프록시: L7까지의 기능을 담당함. 클라이언트가 웹 서비스에 요청을 할 때 프록시를 사용하여 중간에서 프록시가 요청을 대신 전달하는 방식.  속도가 느려지는 단점이 존재함.
- 로깅(Logging)과 감사추적(Auditing)
    - 허용 또는 거부된 접근에 대한 기록을 유지.
- 인증(Authentication)
    - 메시지 인증, 사용자 인증, 클라이언트 인증을 할 수 있음.
- 데이터 암호화
    - 방화벽에서 다른 방화벽까지 전송되는 데이터를 암호화해서 보내는 것으로 보통 VPN의 기능을 이용.

### pfsense

- 기능 및 특징: freeBSD 기반으로 하는 오픈소스 방화벽. 하드웨어나 VM에 설치하여 사용한다.
- WEB UI 지원으로 손쉬운 방화벽 모니터링, 관리가 가능하다.
- IPSec, 라우팅, 방화벽 기능을 기본으로 제공하며, nmap, snort, bind 등의 패키지를 추가로 설치하여 사용 가능하다.

## pfsense 설치와 설정

- 제공된 ISO 파일을 사용하여 설치
- 새 가상 시스템.  ESXI 6.7 버전. 기타. FreeBSD 11.2 버전(64비트)
- 사양: CPU 1. RAM 1GB. HDD 20GB.
- 씬 프로비저닝
- CD/DVD 드라이브: 데이터스토어 ISO파일
- ID//PW: admin//pfsense

### 네트워크 어탭터(중요)

1. route-fw-net
2. dmz-net
3. dev-net
4. soc-net
5. intra-net

### 설정

1. 약관 동의
2. Install
3. Auto (UFS) Guided Disk Setup
4. GPT
5. 리부팅

### 재부팅 후

1. vlan 세팅 여부: N
2. vmx 설정 → MAC 주소와 비교하여 선택.
    1. WAN: route-fw-net
    2. LAN: soc-net
    3. opt1: dmz-net
    4. opt2: dev-net
    5. opt3: intra-net
3. 인터페이스에서 IP 부여하기.
    1. WAN: 10.0.0.254/24. gateway: 10.0.0.1
        1. DHCP, IPv6, Web Contigurator 설정:  N
    2. LAN: 10.0.100.1/24, gateway 없음.
        1. DHCP, IPv6, Web Contigurator 설정:  N
4. Mint 클라이언트에서 받은 URL(https://10.0.100.1)로 접속하기.
    1. Mint 네트워크 어댑터가 soc-net으로 되어 있는지 확인
    2. 터미널에서 sudo xed /etc/netplan/1-network-manger-all.yaml 명령어로 정보 변경하기.
    
    ```c
    sudo xed /etc/netplan/1-network-manger-all.yaml  // 정보 변경
    
    '''
    network:
      version: 2
      renderer: NetworkManger
     
    Network:
      ethernets:
        ens160:
          address: [10.0.100.2/24]
          gateway4: 10.0.100.1
          nameservers:
            address: [8.8.8.8,8.8.4.4]
    '''
    //설정 후
    sudo netplan apply
    	ip addr // 변경 확
    ```
    
5. GUI 인터페이스에 접속 (Mint에서 인터넷 창에 https://10.0.100.1로 접속하기)
6. 고급 → 위험을 감수하고 계속
7.  ID//PW: admin//pfsense

### pfsense wizard 설정

- GUI 접속 후 진행
- Primary DNS Server: 8.8.8.8
- Secondary DNS server: 8.8.4.4
- time server: Asia/Seoul
- Finish 후 대시보드에서 설정 확인

### 인터페이스 설정

- 네트워크 설정: vmx → soc-net 과 같이 이름 변경 및 IP 설정.
    - DMZ: 10.0.10.1/24
    - Dev: 10.0.150.1/24
    - intra: 10.0.200.1/24
- **설정 후 Enable interface 체크**
- **apply 필수**